# Walkthrough for Jupyter

In this notebook we'll walk through how to get rolling with Notebooks.

Important to note that **Jupyter Notebook file ≠ a Python file.**

## 1. Generating the data

For this demo, lets generate two distinct clusters of (x,y,z) coordinate data.

In [ ]:
# BAD CODE, DON"T DO THIS:
import numpy as np
total_numbers = 200
mean1, std_dev1 = 0, 4
cluster1 = np.random.normal(mean1, std_dev1, size=(total_numbers,3))
mean2, std_dev2 = 10, 5
cluster2 = np.random.normal(mean2, std_dev2, size=(total_numbers,3))















In [ ]:
# Preamble:
%matplotlib inline
import numpy as np
import numpy.random as rnd
import pickle as pkl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import tensorflow as tf
from sklearn import model_selection
from IPython.display import clear_output

py.offline.init_notebook_mode(connected=True)

# Global Vars:
total_numbers = 200
number_of_clusters = 2
mean, std_dev = [0, 10], [4, 5]

# Set a seed:
rnd.seed(42) 

# Logging levels:
#tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
def generate_data():
    "This function generates <number_of_clusters> set of x,y,z points"
    global data_points
    
    data_points = []
    
    for group in range(number_of_clusters):
        random_points = rnd.normal(mean[group], std_dev[group], size=(total_numbers,3))
        data_points.append(random_points)

In [ ]:
generate_data()

Now that we have data generated in `data_points`, we don't want to re-generate it everytime. It takes time to compute, and when randomness is at play, we might get different numbers every time. Getting different numbers everytime means we won't know if things have changed for the better. 

**Use seeds!** They are critical in being able to have reproductivity in your code. All variables which deal with randomness and machine learning have a seed option that can be set.

When data has been generated in python, it doesn't need to be re-generated everytime. The `pickle` package is your best friend!

In [ ]:
def save_pickle_file(name, data):
    "Save out the data as a pickle file"
    with open("output/{}.pkl".format(name), "wb") as pickle_file:
        pkl.dump(data, pickle_file)

def load_pickle_file(name):
    "Get the value from the pickle file and set it as the variable value"
    with open("output/{}.pkl".format(name), "rb") as pickle_file:        
        globals()[name] = np.array(pkl.load(pickle_file))

In [ ]:
#save_pickle_file("data_points", data_points)
load_pickle_file("data_points")

## 2. Visualize what we have

Visualizations are one of the key features of Jupyter Notebooks. 

I personally like jot down the type of visualization I am aiming for on paper.

![drawn graph](images/drawing.jpg "Drawing")

In [ ]:
# %load snippets/3dplot.py
marks_and_colors = ["r", "o"], ["b", "^"]

def plot_3d(new_point=False):
    "Makes a 3d plot for a given dataset"
    fig = plt.figure(figsize=(16, 16)) # easy way to get bigger figures
    ax = fig.add_subplot(111, projection='3d')
    
    # Scatter points:
    i = 0
    for group in data_points:
        for point in group:
            ax.scatter(point[0], point[1], point[2], c=marks_and_colors[i][0], marker=marks_and_colors[i][1])
        i = i + 1

    # Check to see if new_point is set:
    if new_point:
        ax.scatter(new_point[0], new_point[1], new_point[2], c="g", marker="o", s=600)
        
    # Labels:
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()
    
    # Save the image:
    fig.savefig('output/3d-plot.png')
    plt.close(fig)

In [ ]:
plot_3d()

Cool! We got the visualization that we wanted, and we save it out everytime it is rendered. Saving the image out and putting it on version control (`git`) is a great way to track the graphics.

### Interactive Visualizations 

Interactive visualizations are not much harder than the interactive one. For our particular case, we will be using [Plot.ly](https://plot.ly/). The service requires you to create an account and have an API key (follow [these instructions](https://plot.ly/python/getting-started/)). 

Even though Plot.ly is a paid service, I have a little life-hack which allows you to create as many plots as you want while sticking to the free plan.

After creating an account with Plot.ly, run the following code one time:
```python
import plotly
plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
```

In [ ]:
# %load snippets/3d-interactive-plot.py
def interactive_3d_plot(new_point=False, save=False):
    data = []
    
    # Marker and color info 
    marks_and_interactive = ["red", "circle"], ["blue", "diamond"]
    
    # Scatter points:
    i = 0
    for g in data_points:
        group = go.Scatter3d(
            x=g[:,0],
            y=g[:,1],
            z=g[:,2],
            mode='markers',
            name='Grouping {}'.format(i+1),
            marker=dict(
                color=marks_and_interactive[i][0],
                size=12,
                symbol=marks_and_interactive[i][1],
                line=dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.2
                ),
                opacity=0.6
            )
        )
        data.append(group)
        i = i + 1
    
    # The layout for the plot:
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    
    # To save the plot, pass in a flag and then render the plot out as a .html file:
    if save:
        file_name = "3d-interactive-plot"
        py.offline.plot({"data": data, "layout": layout}, 
                            auto_open=False, filename="output/{}.html".format(file_name))
        print("Saved the file {}".format(file_name))
    
    # Use the offline plot to see the plot:
    py.offline.iplot({"data":data, "layout":layout})

In [ ]:
interactive_3d_plot(save=True)

## 3. Machine Learning

Let's create a basic `tensorflow` model which is able to classify any given point as either group 1 (red group) and group 2 (blue group).

In [ ]:
# %load snippets/ml.py
def deep_classifier():
    '''
    Make a basic tensorflow DNN
    '''
    # Get the data into an np array which makes things easy
    data = np.concatenate((data_points[0], data_points[1]), axis=0)
    labels = np.repeat([0, 1], 200)
    
    # Split into training and testing data:
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        data, labels, test_size=0.2, random_state=100)
    
    feature_columns = [tf.feature_column.numeric_column('x', shape=np.array(x_train).shape[1:])] 
    
    get_train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x':x_train}, y=y_train, num_epochs=5, shuffle=True)
    
    get_test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x':x_test}, y=y_test, num_epochs=1, shuffle=True)
    
    # Create a DNN classifier:
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
            hidden_units=[1000, 100, 10], n_classes=2, model_dir="model",
            optimizer=tf.train.ProximalAdagradOptimizer(
                learning_rate=0.01,
                l1_regularization_strength=0.01),
                config=tf.estimator.RunConfig().replace(save_summary_steps=10))
    
    # Train the classifier:
    classifier.train(input_fn=get_train_input_fn, steps=1000)

    scores = classifier.evaluate(input_fn=get_test_input_fn)
    
    print('Accuracy (tf.estimator): {0:f}'.format(scores['accuracy']))
    
    return classifier

In [ ]:
classifier = deep_classifier()

Different results can be gotten by changing various parameters of the DNNClassifier (such as the `learning_rate` and `num_epochs`). Since the same directory `model/` is being used to serve TensorBoard, TensorBoard will keep track of historical models.

In [ ]:
def make_prediction(points):
    "Try to predict the class of the point for our own point"
    our_point_fn = tf.estimator.inputs.numpy_input_fn(x={'x':points}, shuffle=False)

    # Make a prediction:
    prediction = classifier.predict(input_fn=our_point_fn)
    return prediction

# The points:
points = np.array([[1, -5, 4], [12, 2, 15]]) # Answers: class 0, then class 1

for index, pred in enumerate(list(make_prediction(points))):
    print(points[index], '→' , pred['class_ids'])

## 4. Interactivity

Most people do not know this but Jupyter has a built in feature which allows us to to easily take in user input. Here we'll take in the `x,y,z` points from the user and then make a classification based on the model prediction. Here are the [docs for interactive widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html).

In [ ]:
from ipywidgets import widgets
%matplotlib notebook

In [ ]:
# Define the x, y, z sliders
x = widgets.FloatSlider(
    value=1.0,
    min=np.min(data_points[0][:, 0]),
    max=np.max(data_points[1][:, 0]),
    step=1.0,
    description='X:',
    continuous_update = False
)

y = widgets.FloatSlider(
    value=1.0,
    min=np.min(data_points[0][:, 1]),
    max=np.max(data_points[1][:, 1]),
    step=1.0,
    description='Y:',
    continuous_update = False
)

z = widgets.FloatSlider(
    value=1.0,
    min=np.min(data_points[0][:, 2]),
    max=np.max(data_points[1][:, 2]),
    step=1.0,
    description='Z:',
    continuous_update = False
)

button = widgets.Button(description="Predict Point")

In [ ]:
box = widgets.HBox(children=[x, y, z, button])

In [ ]:
fig = plt.figure(figsize=(13, 6))
fig.canvas.set_window_title('Prediction Maker')
ax = fig.add_subplot(121, projection='3d')
ay = fig.add_subplot(122)
ax.set_title('Points in Space')

def create_initial_plot():
    "Create the initial plot with all the data_points"
    # Add the points: 
    i = 0
    for group in data_points:
        for point in group:
            ax.scatter(point[0], point[1], point[2], c=marks_and_colors[i][0], marker=marks_and_colors[i][1])
        i = i + 1

    # Labels:
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

def button_clicked(change):
    "Ran when the button is clicked"
    global my_point
    # Remove the old point:
    try:
        my_point.remove()
    except:
        print("The green point has been added")
        
    # Plot our new point:
    my_point = ax.scatter(x.value, y.value, z.value, c="g", marker="o", s=600)
    
    # Make the prediciton:
    prediction = make_prediction(np.array([[x.value, y.value, z.value]]))
    prob = list(prediction)[0]['probabilities']
    
    # Plot the prediction:
    ay.cla()
    ay.set_title('Point Prediction')
    ay.bar([0, 1], prob, align='center', color=['red', 'blue'])
    ay.set_xticks([0, 1])
    ay.set_xticklabels(['red', 'blue'])
    
create_initial_plot()

In [ ]:
# Listen for when the button is clicked, then make the callback:
button.on_click(button_clicked)

# Display the box:
display(box)

In order to listen for just changes in specific values (without a button), use the following: `x.observe(some_function)` where `some_function` is a callback function. 

### Useful magic commands:

Magic commands are run by havinv `%` at the start of a cell. Here are some useful ones I use often:

- `%time` - Times execution of a cell.
- `%load hello_world.py` - Will load in the contents of a file into a cell
- `%store data` - Will store data to be used by other Notebooks. `%store -r data` in a different Notebook will bring in that data to be used.
- `%lsmagic` - List out all the magic commands.

To learn about any magic command, run the magic command with a `?` at the end. For instance `%time?`.

You can also run bash commands by having `!` before a bash command. For instance `!ls`.

In [ ]:
!wget

In [ ]:
%time
print('time me')

In [ ]:
%lsmagic

In [ ]:
%%latex
$y=mx+b$

In [ ]:
%%ruby
puts 'Ruby inside Jupyter!'

## Jupyter to Python conversion:

```
jupyter nbconvert --to script Test.ipynb
```

Make some minor changes to the `.py` file:

1. Get rid of the magic line
2. Add the following line before you import plt `matplotlib.use('agg')`
3. Instead of `plt.show()` use `plt.savefig('output/languages.png')`
4. Run with $ python3 <file>.py

## Make Slides with Jupyter:

```
jupyter nbconvert --to slides Slides.ipynb --reveal-prefix "https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.6.0"
```

In [ ]:
%%html
<h3>If you have any questions, feel free to <a href="mailto:mnkhan@ucdavis.edu">email me</a>!</h3>